In [10]:
import os
import pandas as pd
from clickhouse_driver import Client
import logging
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования с указанием кодировки
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("load_excel_to_clickhouse.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Каталог с Excel файлами
excel_directory = './Агрегат'

# Создание ClickHouse client с указанием настроек
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

# Определение ожидаемых типов данных для каждого столбца
expected_column_types = {
    'ComponentNumber': 'Nullable(String)',
    'UnloadDate': 'Nullable(Date)',
    'NR_Mi8T': 'Nullable(Float64)',
    'MRR_Mi8T': 'Nullable(Float64)',
    'NR_Mi17': 'Nullable(Float64)',
    'MRR_Mi17': 'Nullable(Float64)',
    'MRR_2_Mi8T': 'Nullable(Float64)',
    'MRR_2_Mi17': 'Nullable(Float64)',
    'RepairTime': 'Nullable(Float64)',
    'PurchasePrice': 'Nullable(Float64)',
    'RepairPrice': 'Nullable(Float64)',
    'BR_Mi8T': 'Nullable(Float64)',
    'BR_Mi17': 'Nullable(Float64)',
    'items_per_ac': 'Nullable(Float64)'
}

def convert_value(key, value):
    if key == 'UnloadDate':
        # Преобразуем значение в дату
        try:
            return pd.to_datetime(value).date()
        except Exception:
            return None
    elif key in ['BR_Mi8T', 'BR_Mi17', 'NR_Mi8T', 'MRR_Mi8T', 'NR_Mi17', 'MRR_Mi17', 'MRR_2_Mi8T', 'MRR_2_Mi17', 'RepairTime', 'PurchasePrice', 'RepairPrice', 'items_per_ac']:
        # Преобразуем значение в float
        try:
            return float(value)
        except (ValueError, TypeError):
            return None
    else:
        # Для остальных строковых полей
        if isinstance(value, str):
            return value.strip()
        return value

def load_excel_to_clickhouse(file_path, table_name):
    try:
        # Прочитать Excel файл без заголовков
        df = pd.read_excel(file_path, engine='openpyxl', header=None)
        logger.info(f"Файл {file_path} успешно прочитан.")

        # Предполагая, что первый столбец - названия полей, второй - значения
        field_names = df.iloc[:, 0].astype(str).apply(lambda x: x.strip())
        values = df.iloc[:, 1]

        # Создать словарь данных
        data_dict = dict(zip(field_names, values))

        # Обработка кодировки UTF-8 для названий полей и строковых значений
        data_dict_cleaned = {}
        for key, value in data_dict.items():
            clean_key = key.encode('utf-8', errors='replace').decode('utf-8', errors='replace')
            if isinstance(value, str):
                clean_value = value.encode('utf-8', errors='replace').decode('utf-8', errors='replace')
            elif value is None or pd.isnull(value):
                clean_value = None
            else:
                clean_value = value
            data_dict_cleaned[clean_key] = clean_value

        # Замена русских названий таблицы и столбцов на английские
        table_name_ascii = 'Agregat'  # Используем ASCII имя таблицы
        data_dict_ascii = {}
        column_mapping = {
            'Чертежный номер компонента': 'ComponentNumber',
            'Дата выгрузки': 'UnloadDate',
            'НР Ми-8Т': 'NR_Mi8T',
            'МРР Ми-8Т': 'MRR_Mi8T',
            'НР Ми-17': 'NR_Mi17',
            'МРР Ми-17': 'MRR_Mi17',
            'МРР 2 Ми-8Т': 'MRR_2_Mi8T',
            'МРР 2 Ми-17': 'MRR_2_Mi17',
            'Срок ремонта': 'RepairTime',
            'Цена покупки': 'PurchasePrice',
            'Цена ремонта': 'RepairPrice',
            'BR Ми-8Т': 'BR_Mi8T',
            'BR Ми-17': 'BR_Mi17',
            'Колво на ВС': 'items_per_ac'
        }

        for key, value in data_dict_cleaned.items():
            ascii_key = column_mapping.get(key, key)
            data_dict_ascii[ascii_key] = value

        # Преобразование значений в соответствии с ожидаемыми типами
        data_dict_converted = {}
        for key, value in data_dict_ascii.items():
            if key in expected_column_types:
                data_dict_converted[key] = convert_value(key, value)
            else:
                data_dict_converted[key] = value  # Если ключ не в expected_column_types, оставляем как есть

        # Создание таблицы с явными типами данных
        columns_with_types = [f'`{key}` {expected_column_types[key]}' for key in expected_column_types]

        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {database_name}.{table_name_ascii} (
            {', '.join(columns_with_types)}
        ) ENGINE = MergeTree()
        ORDER BY tuple()
        """
        logger.info(f"CREATE TABLE Query:\n{create_table_query}")
        client.execute(create_table_query)
        logger.info(f"Таблица {table_name_ascii} создана или уже существует.")

        # Подготовка данных для вставки
        columns = list(expected_column_types.keys())
        values = []
        for col in columns:
            val = data_dict_converted.get(col)
            if 'String' in expected_column_types.get(col, ''):
                if val is None:
                    val = ''  # Заменяем None на пустую строку для строковых столбцов
            values.append(val)

        # Вставка данных в таблицу
        insert_query = f"INSERT INTO {database_name}.{table_name_ascii} ({', '.join(f'`{col}`' for col in columns)}) VALUES"
        client.execute(insert_query, [values], types_check=True)
        logger.info(f"Данные из файла {os.path.basename(file_path)} успешно загружены в таблицу {table_name_ascii} в ClickHouse.")

    except Exception as e:
        logger.error(f"Ошибка при загрузке файла {os.path.basename(file_path)}: {e}", exc_info=True)
        if hasattr(e, 'code'):
            logger.error(f"Error code: {e.code}")
        if hasattr(e, 'message'):
            logger.error(f"Error message: {e.message}")

def main():
    if not os.path.isdir(excel_directory):
        logger.error(f"Каталог {excel_directory} не существует.")
        return

    files = [f for f in os.listdir(excel_directory) if f.endswith(('.xlsx', '.xls'))]
    if not files:
        logger.info(f"В каталоге {excel_directory} нет Excel файлов для обработки.")
        return

    for file_name in files:
        file_path = os.path.join(excel_directory, file_name)
        logger.info(f"Начинаю обработку файла: {file_path}")
        load_excel_to_clickhouse(file_path, 'Agregat')

if __name__ == "__main__":
    main()


2024-11-26 12:40:28,491 - INFO - Начинаю обработку файла: ./Агрегат/Агрегат.xlsx
2024-11-26 12:40:28,504 - INFO - Файл ./Агрегат/Агрегат.xlsx успешно прочитан.
2024-11-26 12:40:28,506 - INFO - CREATE TABLE Query:

        CREATE TABLE IF NOT EXISTS default.Agregat (
            `ComponentNumber` Nullable(String), `UnloadDate` Nullable(Date), `NR_Mi8T` Nullable(Float64), `MRR_Mi8T` Nullable(Float64), `NR_Mi17` Nullable(Float64), `MRR_Mi17` Nullable(Float64), `MRR_2_Mi8T` Nullable(Float64), `MRR_2_Mi17` Nullable(Float64), `RepairTime` Nullable(Float64), `PurchasePrice` Nullable(Float64), `RepairPrice` Nullable(Float64), `BR_Mi8T` Nullable(Float64), `BR_Mi17` Nullable(Float64), `items_per_ac` Nullable(Float64)
        ) ENGINE = MergeTree()
        ORDER BY tuple()
        
2024-11-26 12:40:28,518 - INFO - Таблица Agregat создана или уже существует.
2024-11-26 12:40:28,525 - INFO - Данные из файла Агрегат.xlsx успешно загружены в таблицу Agregat в ClickHouse.
